<div style="background-color: #cfc ; padding: 20px; border-radius: 10px ; border: 2px solid green;">
<p><font size="+3"><b><center> European Option Pricing </center></b></font>
</p>
<p>
<font size="3px">  <center> A demonstration of the Forge Monte Carlo API.</center></font>
</p>    
</div>

<div class="alert alert-block alert-warning">
    Note: The following notebook uses features of Forge (data loaders) which are not available to Developer-level (free) accounts.
</div>

**forge.montecarlo** is Forge's API for Quantum Monte Carlo simulations.

Our ambition is to deliver the first real applications of quantitative finance with Quantum Monte Carlo simulations. The following notebook will show you how to price a European option with quantum computers using Forge.

This notebook leverages european_option.py, which defines the circuits for option pricing.

## Import QCWare libraries

In [1]:
import quasar
import qcware
import numpy as np
from qcware.forge import qutils
from european_option import EuropeanOption

## Define the EuropeanOption
<div class="alert alert-block alert-warning">
The call to EuropeanOption below requires a paid account due to its use of qio.loader.
</div>

In [2]:
epsilon = 0.005             # Precision parameter for the estimate
n_asset_prices = 8          # Asset price distribution will be discretized in 8 parts. This means we'll use 8 qubits to simulate prices, and 1 qubit to compute payoff. 

initial_asset_price = 100
strike_price = 110
interest_rate = 0.05
volatility = 0.1
time_to_simulate = 1        # Time to simulate in years
option_type = 'C'           # Call option

new_product = EuropeanOption(n_asset_prices, initial_asset_price, strike_price, interest_rate, volatility, time_to_simulate, option_type, epsilon)

#### Price with Black-Scholes formula

This is a closed-form solution that makes several assumptions about the market.

In [3]:
print("Price with Black Scholes formula:",round(new_product.bsm,3))
print(new_product.asset_distribution)

Price with Black Scholes formula: 2.174
[ 84.04901448  90.07132738  96.09364029 102.11595319 108.13826609
 114.16057899 120.18289189 126.20520479]


#### Run a perfect simulation of the quantum pricing circuit on a CPU simulator

Note: we're limited in precision by the fact that we're discretizing the asset price distribution into 8 parts, so we can't hope to match the Black-Scholes estimate exactly.

In [4]:
perfect_price = new_product.price(n_shots=None)
print('Perfect simulation price after discretization:',perfect_price)

Perfect simulation price after discretization: 1.9441536619918698


#### What's the goal of the montecarlo API?
We just ran a perfect simulation of the quantum pricing algorithm on the cpu simulator, taking n_shots = infinity. Take note of this result.

We'll now try to get close to this result with <b>far fewer runs of the model (fewer "oracle calls")</b> using the quantum monte carlo API.

Here's our first attempt, using all the default parameters:

In [5]:
quantum_price = new_product.price()
print('Estimated price:', quantum_price)
print('Shots:', new_product.shots)
print('Model runs:', new_product.oracle_calls)

Estimated price: 1.9413351971712158
Shots: 110
Model runs: 1210


## Going deeper into the quantum Monte Carlo method

Let us provide a high level description of the quantum Monte Carlo method.

We start with the model, which is a quantum circuit which we can measure directly to get a sample. For example here, the quantum circuit creates a distribution over asset prices and then computes the payoff function for each of them in superposition, so that when we measure we get a sample from the distribution of the expected payoff. 

The main idea is that not only do we have the possibility to sample from this quantum circuit directly, but we can create deeper quantum circuits by repeating this quantum model sequentially, so that sampling from all these quantum circuits with different depths and cleverly combining all the results, reduces the total number of samples. 

Thus, the first important part of the quantum Monte Carlo method is to define a schedule of which circuits to samples from and how many times. In other words, we need to define a schedule, which is a list of pairs {($D_1$,$N_1$), ($D_2$,$N_2$),...($D_k$,$N_k$)}, that tells the quantum algorithm to run each quantum circuit of depth $D_i$ for $N_i$ shots. Different schedules give different accuracies with different number of samples!

Here, we have some predefined types of schedules ('linear','exponential','powerlaw,'direct') one can use or one can define their own schedule by just providing as a list.

### Schedules for quantum pricing

We can use different types of schedules for the quantum pricing algorithm.

#### Examples of schedules
 - (schedule_type='direct', n_shots=10000):<br/>
         -> [[0,10000]]
 - (schedule_type='linear', max_depth=10,n_shots=10):<br/>
         -> [[0,10],[1,10],[2,10],[3,10],[4,10],...,[10,10]]
 - (schedule_type='exponential', max_depth=20,n_shots=10):<br/>
         -> [[1,10],[2,10],[4,10],[8,10],[16,10]]
 - schedule_type='powerlaw', beta=0.15, n_shots=10):<br/>
         -> [[0, 10], [6, 10], [21, 10], [49, 10], [94, 10]]
   - This is a more complicated schedule whose max depth depends on beta.
   - Note, beta can result in very large computations.


#### Print the schedule used in the previous example

In [6]:
print(new_product.schedule)

[[0, 10], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10]]


#### Benchmark different schedules

In [7]:
# 1. perfect simulation
perfect_price = new_product.price(n_shots=None)
print('Perfect price after discretization:',perfect_price)

# 2. direct schedule (no amplitude estimation)
direct_price = new_product.price(schedule_type='direct', n_shots=100000)
print('\nDirect Schedule (no amplitude estimation)')
print(' Estimated price:', direct_price)
print(' Model runs:', new_product.oracle_calls)

# 3. linear schedule
quantum_price = new_product.price(schedule_type='linear', max_depth=10, n_shots=10)
print('\nLinear Schedule')
print(' Estimated price:', quantum_price)
print(' Model runs:', new_product.oracle_calls)

# 3. exponential schedule
quantum_price = new_product.price(schedule_type='exponential', max_depth=20, n_shots=10)
print('\nExponential Schedule')
print(' Estimated price:', quantum_price)
print(' Model runs:', new_product.oracle_calls)

# 4. powerlaw schedule,
quantum_price = new_product.price(schedule_type='powerlaw', n_shots=10, beta=0.8)
print('\nPowerlaw Schedule')
print(' Estimated price:', quantum_price)
print(' Model runs:', new_product.oracle_calls)


Perfect price after discretization: 1.9441536619918698



Direct Schedule (no amplitude estimation)
 Estimated price: 1.9413351971712158
 Model runs: 100000



Linear Schedule
 Estimated price: 1.9413351971712158
 Model runs: 1210



Exponential Schedule
 Estimated price: 1.9413351971712158
 Model runs: 670



Powerlaw Schedule
 Estimated price: 1.9413351971712158
 Model runs: 139030


### Looking into the quantum circuits

We can check the quantum circuits that correspond to the model (initial_circuit) and to the iteration circuit sequentially in the quantum Monte Carlo method (iteration_circuit)

In [8]:
print('Initial circuit – Oracle\n')
print(new_product.initial_circuit)

Initial circuit – Oracle

T  : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|

q0 : -X-B-B-B----------------------------------
        | | |                                  
q1 : ---|-|-S----------------------------------
        | |                                    
q2 : ---|-S-B----------------------------------
        |   |                                  
q3 : ---|---S----------------------------------
        |                                      
q4 : ---S-B-B----------------------------------
          | |                                  
q5 : -----|-S----@----@------------------------
          |      |    |                        
q6 : -----S-B----|----|----@----@--------------
            |    |    |    |    |              
q7 : -------S----|----|----|----|-----@-----@--
                 |    |    |    |     |     |  
q8 : ---------Ry-X-Ry-X-Ry-X-Ry-X--Ry-X--Ry-X--
                                               
T  : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|



In [9]:
print('Iteration circuit \n')
print(new_product.iteration_circuit)

Iteration circuit 

T  : |0|1|2 |3|4 |5|6 |7|8 |9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|

q0 : ----------------------------------B--B--B--@--Z--B--B--B-----------------
                                       |  |  |  |     |  |  |                 
q1 : ----------------------------------S--|--|--|-----|--|--S-----------------
                                          |  |  |     |  |                    
q2 : ----------------------------------B--S--|--|-----|--S--B-----------------
                                       |     |  |     |     |                 
q3 : ----------------------------------S-----|--|-----|-----S-----------------
                                             |  |     |                       
q4 : ----------------------------------B--B--S--|-----S--B--B-----------------
                                       |  |     |        |  |                 
q5 : -----------------------@----@-----S--|-----|--------|--S-----@-----@-----
                            |  

#### Using parallel quantum circuits

We can also use a different type of quantum circuit that uses more qubits has less depth.

In [10]:
# one optional parameter for optimizing depth vs qubits
mode = 'parallel'   # 'parallel' = optimized depth, 'sequential' = optimized number of qubits

# Define a new European Option product
new_product_parallel = EuropeanOption(n_asset_prices, initial_asset_price, strike_price, interest_rate, volatility, time_to_simulate, option_type, epsilon, mode=mode)
new_product_parallel.price()

1.9413351971712158

In [11]:
print(new_product_parallel.initial_circuit)

T   : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|

q0  : -X-B-B-B----------------------------------
         | | |                                  
q1  : ---|-|-S----------------------------------
         | |                                    
q2  : ---|-S-B----------------------------------
         |   |                                  
q3  : ---|---S----------------------------------
         |                                      
q4  : ---S-B-B----------------------------------
           | |                                  
q5  : -----|-S----@----@------------------------
           |      |    |                        
q6  : -----S-B----|----|----@----@--------------
             |    |    |    |    |              
q7  : -------S----|----|----|----|-----@-----@--
                  |    |    |    |     |     |  
q8  : ---------Ry-X-Ry-X----|----|-----|-----|--
                            |    |     |     |  
q9  : -------------------Ry-X-Ry-X-----|-----|--
                   

In [12]:
print(new_product_parallel.iteration_circuit)

T   : |0|1|2 |3|4 |5|6 |7|8 |9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|

q0  : ----------------------------------B--B--B--@--@--@--Z--B--B--B-----------
                                        |  |  |  |  |  |     |  |  |           
q1  : ----------------------------------S--|--|--|--|--|-----|--|--S-----------
                                           |  |  |  |  |     |  |              
q2  : ----------------------------------B--S--|--|--|--|-----|--S--B-----------
                                        |     |  |  |  |     |     |           
q3  : ----------------------------------S-----|--|--|--|-----|-----S-----------
                                              |  |  |  |     |                 
q4  : ----------------------------------B--B--S--|--|--|-----S--B--B-----------
                                        |  |     |  |  |        |  |           
q5  : -----------------------@----@-----S--|-----|--|--|--------|--S-----@-----
                             |    |    

#### Defining your own quantum circuits

You can define your own circuits for applying the quantum Monte Carlo methods. More details on the NISQ Amplitude Estimation notebook